In [210]:
from manim import *
import math
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching ndw_interference_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s ndw_interference_Scene"

In [213]:
background_color = "#20455A"

background_teint_shade_dict = {
    -10: "#ffffff",
    -9: "#e8ecee",
    -8: "#d2d9de",
    -7: "#bcc7cd",
    -6: "#a5b4bd",
    -5: "#8fa2ac",
    -4: "#798f9c",
    -3: "#627c8b",
    -2: "#4c6a7a",
    -1: "#36576a",
    0: "#20455A",
    1: "#1c3e51",
    2: "#193748",
    3: "#16303e",
    4: "#132936",
    5: "#10222d",
    6: "#0c1b24",
    7: "#09141b",
    8: "#060d12",
    9: "#030609",
    10: "#000000",
}


# returns 
def color_mixer(amplitude):
    amplitude -= np.sign(amplitude) / 1000

    # colors
    int_up = int(10*amplitude + np.sign(amplitude))
    int_down = int(10*amplitude)

    color_up = background_teint_shade_dict[int_up]
    color_down = background_teint_shade_dict[int_down]

    # opacities
    opacity_up = abs(amplitude) % 1
    opacity_down = 1 - opacity_up
    return ( (color_down, opacity_down*abs(amplitude)), (color_up, opacity_up*abs(amplitude)) )


# frequency over time
def nu_smooth(t):
    return 200 * (2 + math.erf(2/5*(t-21)))

In [215]:
class SoundSources(Mobject):
    def __init__(self, center_A = np.array([-2.5, -1.5, 0]), center_B = np.array([2.5, -1.5, 0]), c = 300, r0 = 0.75, wave_density = 40, wave_width = 5, **kwargs):
        super().__init__(**kwargs)

        self.speaker_A = center_A                       # center of speaker A
        self.speaker_B = center_B                       # center of speaker B

        self.c = c                                      # speed of sound
        self.r0 = r0                                    # minimal distance from the sound sources
        self.d = np.sqrt(sum((center_A-center_B)**2))        # distance between the sound sources
        self.wave_density = wave_density                # number wavelines for for dr = 1
        self.circle_stroke_width = wave_width

        # place the speaker images (definitely violating some copyrights on the way)
        speaker_bg_left = Circle(radius = 0.985*self.r0, color = background_color, stroke_width = 2, stroke_opacity = 0, fill_opacity = 1).move_to(self.speaker_A)
        speaker_bg_right = Circle(radius = 0.985*self.r0, color = background_color, stroke_width = 2, stroke_opacity = 0, fill_opacity = 1).move_to(self.speaker_B)
        speaker_bg_left.z_index = 10
        speaker_bg_right.z_index = 10

        speaker_left = ImageMobject("../external_media/lautsprecher_black.png").scale(0.25).move_to(center_A)
        speaker_right = ImageMobject("../external_media/lautsprecher_black.png").scale(0.25).move_to(center_B)
        speaker_left.z_index = 15
        speaker_right.z_index = 15
        self.add(speaker_bg_left, speaker_bg_right, speaker_left, speaker_right)


    # calculates the amplitude of the wave depending on frequency nu, distance from the source r, and time t
    def get_amplitude(self, nu, r, t):
        omega = 2*PI*nu                                 # circle frequency
        k = omega / self.c                              # wave number
        a0 = min(k * self.d / 4, k*self.r0)             # a0 set to limit amplitude to 1
        return a0 / k / r * (np.sin(omega*t - k*r))
    
    
    # returns the wave field as a group of circles around both sound sources
    def get_sound_waves(self, nu, t):
        # condition for waves: distance bigger than minimum, distance lower than speed of sound times time
        r = self.r0                                     # set r to minimum spread              
        r_max = self.c * t                              # set r_max to maximum spread
        dr = 1.0 / self.wave_density                    # radius increment 
        wave_group = VGroup()                           # group with all wave lines
        # r_max line
        if t > self.r0 / self.c:
            amplitude = self.get_amplitude(nu, r_max, t)
            # line_down_color, line_up_color = color_mixer(amplitude)
            # wave_A_up_color = Circle(radius = r_max, color = line_up_color[0], stroke_width = self.circle_stroke_width, stroke_opacity = 0.25).move_to(self.speaker_A)
            # wave_A_down_color = Circle(radius = r_max, color = line_down_color[0], stroke_width = self.circle_stroke_width, stroke_opacity = 0.25).move_to(self.speaker_A)
            # wave_B_up_color = Circle(radius = r_max, color = line_up_color[0], stroke_width = self.circle_stroke_width, stroke_opacity = 0.25).move_to(self.speaker_B)
            # wave_B_down_color = Circle(radius = r_max, color = line_down_color[0], stroke_width = self.circle_stroke_width, stroke_opacity = 0.25).move_to(self.speaker_B)
            # wave_group.add(wave_A_up_color, wave_A_down_color, wave_B_up_color, wave_B_down_color)
            if amplitude > 0:
                wave_A = Circle(radius = r_max, color = BLACK, stroke_width = self.circle_stroke_width, stroke_opacity = amplitude).move_to(self.speaker_A)
                wave_B = Circle(radius = r_max, color = BLACK, stroke_width = self.circle_stroke_width, stroke_opacity = amplitude).move_to(self.speaker_B)
            else:
                wave_A = Circle(radius = r_max, color = WHITE, stroke_width = self.circle_stroke_width, stroke_opacity = abs(amplitude)).move_to(self.speaker_A)
                wave_B = Circle(radius = r_max, color = WHITE, stroke_width = self.circle_stroke_width, stroke_opacity = abs(amplitude)).move_to(self.speaker_B)
            wave_A.z_index = -5
            wave_B.z_index = -5
            wave_group.add(wave_A, wave_B)
        # all other radius lines
        while r < min(r_max, 12):
            amplitude = self.get_amplitude(nu, r, t)    # amplitude of the waves (between 0 and 1)
            # line_down_color, line_up_color = color_mixer(amplitude)
            # wave_A_up_color = Circle(radius = r, color = line_up_color[0], stroke_width = self.circle_stroke_width, stroke_opacity = 0.2).move_to(self.speaker_A)
            # wave_A_down_color = Circle(radius = r, color = line_down_color[0], stroke_width = self.circle_stroke_width, stroke_opacity = 0.25).move_to(self.speaker_A)
            # wave_B_up_color = Circle(radius = r, color = line_up_color[0], stroke_width = self.circle_stroke_width, stroke_opacity = 0.25).move_to(self.speaker_B)
            # wave_B_down_color = Circle(radius = r, color = line_down_color[0], stroke_width = self.circle_stroke_width, stroke_opacity = 0.25).move_to(self.speaker_B)
            # wave_group.add(wave_A_up_color, wave_A_down_color, wave_B_up_color, wave_B_down_color)
            if amplitude > 0:
                wave_A = Circle(radius = r, color = BLACK, stroke_width = self.circle_stroke_width, stroke_opacity = amplitude).move_to(self.speaker_A)
                wave_B = Circle(radius = r, color = BLACK, stroke_width = self.circle_stroke_width, stroke_opacity = amplitude).move_to(self.speaker_B)
            else:
                wave_A = Circle(radius = r, color = WHITE, stroke_width = self.circle_stroke_width, stroke_opacity = abs(amplitude)).move_to(self.speaker_A)
                wave_B = Circle(radius = r, color = WHITE, stroke_width = self.circle_stroke_width, stroke_opacity = abs(amplitude)).move_to(self.speaker_B)
            wave_A.z_index = -5
            wave_B.z_index = -5
            wave_group.add(wave_A, wave_B)
            r += dr
        return wave_group
    

    # draws a hyperbola from vertex distance a from center between speakers
    def get_hyperbola(self, a, color_hyperbola):
        (x0, y0, z0) = (self.speaker_A + self.speaker_B) / 2                # center between the 2 speakers
        c = self.d / 2                                                      # focal point distance to center between speakers
        b = np.sqrt(c**2 - a**2)                                            # co-vertex distance a from center between speakers
        def hyperbola(x, y):
            smoothing_factor = 10e-5
            return (x-x0)**2 / (a+smoothing_factor)**2 - (y-y0)**2 / (b+smoothing_factor)**2 - 1
        return ImplicitFunction(hyperbola, color = color_hyperbola, stroke_opacity = 1, stroke_width = 8)
    

    # return hyperbolas of constructive interrference
    def get_hyperbola_constructive(self, nu):
        lambda_wavelength = self.c / nu
        constructive_hyperbola_group = VGroup()
        for i in range(int(self.d / lambda_wavelength + 0.6)):
            a = lambda_wavelength / 2 * i
            constructive_hyperbola = self.get_hyperbola(a, RED)
            constructive_hyperbola.z_index = 1
            constructive_hyperbola_group.add(constructive_hyperbola)
        return constructive_hyperbola_group
    

    # return hyperbolas of destructive interrference
    def get_hyperbola_destructive(self, nu):
        lambda_wavelength = self.c / nu
        destructive_hyperbola_group = VGroup()
        for i in range(int(self.d / lambda_wavelength + 1.1)):
            a = lambda_wavelength / 4 * (2*i - 1)
            destructive_hyperbola = self.get_hyperbola(a, BLUE)
            destructive_hyperbola.z_index = 1
            destructive_hyperbola_group.add(destructive_hyperbola)
        return destructive_hyperbola_group
    

    # add a legend for the interference
    def get_hyperbola_legend(self, opacity = 0.95):
        legend_bg = Rectangle(height = 1, width = 4, stroke_color = WHITE, stroke_opacity = 0, fill_opacity = opacity).align_on_border(UP + LEFT, buff = 0.25)
        legend_bg.z_index = 3

        line_constructive = Line(start = [0, 0, 0], end = [0.5, 0, 0], color = RED, stroke_opacity = 1, stroke_width = 8).align_on_border(UP + LEFT)
        line_destructive = Line(start = [0, 0, 0], end = [0.5, 0, 0], color = BLUE, stroke_opacity = 1, stroke_width = 8).align_on_border(UP + LEFT).shift(0.5*DOWN)
        line_constructive.z_index = 5
        line_destructive.z_index = 5

        text_constructive = Text("constructive interference", color = RED).scale(0.25).align_on_border(UP + LEFT).shift(RIGHT).shift(0.075*UP)
        text_destructive = Text("destructive interference", color = BLUE).scale(0.25).align_on_border(UP + LEFT).shift(RIGHT + 0.5*DOWN).shift(0.075*UP)
        # whatever this shit is, it works and it only works like this
        for i in range(len(text_constructive)):
            text_constructive[i].z_index = 5
        for i in range(len(text_destructive)):
            text_destructive[i].z_index = 5

        return VGroup(legend_bg, line_constructive, line_destructive, text_constructive, text_destructive)


    # add a nu tracker as legend for the frequency
    def get_nu_legend(self, nu, opacity = 0.95):
        legend_bg = Rectangle(height = 1, width = 4, stroke_color = WHITE, stroke_opacity = 0, fill_opacity = opacity).align_on_border(UP + RIGHT, buff = 0.25)
        legend_bg.z_index = 3

        nu_number_line = NumberLine(x_range = [1, 4], length = 3, color = BLACK).move_to(legend_bg.get_center())
        for tick in nu_number_line.ticks:
            tick.z_index = 5
        nu_number_line.z_index = 5

        # calculate axis position
        # if type(nu) == int or type(nu) == float:
        x_number_line = np.log10(nu/2)                                  # position on the number line
        pos_number_line = nu_number_line.number_to_point(x_number_line)
        nu_line = Line(start = pos_number_line+0.125*UP, end = pos_number_line-0.125*UP, color = RED, stroke_width = 2)
        nu_line.z_index = 10
        nu_text = Text(f"{int(nu)} Hz", color = RED).scale(0.25).next_to(nu_line, 0.5*DOWN)
        for letter in nu_text:
            letter.z_index = 5
        # given a band of frequencies
        # else:
        #     x_min_number_line = min(nu)
        #     x_max_number_line = max(nu)


        return VGroup(legend_bg, nu_number_line, nu_line, nu_text)

In [216]:
%%manim -qh --fps 60 $video_scene


class ndw_interference_Scene(Scene):
    def construct(self):
        self.camera.background_color = background_color

        # parameters
        c = 300
        t0 = 0
        nu = nu_smooth(t0)


        sound_source = SoundSources(c = c, wave_density = 40, wave_width = 5)
        self.add(sound_source)

        sound_waves = sound_source.get_sound_waves(nu, t0/c)
        self.add(sound_waves)


        # constructive interference hyperbolas
        constructive_interference = sound_source.get_hyperbola_constructive(nu)
        destructive_interference = sound_source.get_hyperbola_destructive(nu)
        # self.add(constructive_interference, destructive_interference)


        # legends
        interference_legend = sound_source.get_hyperbola_legend()
        # self.add(interference_legend)

        nu_legend = sound_source.get_nu_legend(nu)
        # self.add(nu_legend)


        def sound_waves_updater(wave):
            t = time_tracker.get_value()
            nu = nu_smooth(t)
            wave.become(sound_source.get_sound_waves(nu, t))


        def constructive_interference_updater(interference):
            t = time_tracker.get_value()
            nu = nu_smooth(t)
            interference.become(sound_source.get_hyperbola_constructive(nu))


        def destructive_interference_updater(interference):
            t = time_tracker.get_value()
            nu = nu_smooth(t)
            interference.become(sound_source.get_hyperbola_destructive(nu))


        def nu_legend_updater(legend):
            t = time_tracker.get_value()
            nu = nu_smooth(t)
            legend.become(sound_source.get_nu_legend(nu))


# +++ PART A (set t0 = 0) +++

        self.wait(1.5)
        time_tracker = ValueTracker(t0/c)

        sound_waves.add_updater(sound_waves_updater)

        self.play(time_tracker.animate.set_value(10/c), rate_func = linear, run_time = 10)
        self.play(time_tracker.animate.set_value(13/c), FadeIn(constructive_interference), FadeIn(destructive_interference), FadeIn(interference_legend), FadeIn(nu_legend), rate_func = linear, run_time = 3)
        self.play(time_tracker.animate.set_value(16/c), rate_func = linear, run_time = 3)


# +++ PART B (set t0 = 16) +++

        # self.add(constructive_interference, destructive_interference)
        # self.add(interference_legend)
        # self.add(nu_legend)

        # time_tracker = ValueTracker(t0/c)

        # sound_waves.add_updater(sound_waves_updater)
        # nu_legend.add_updater(nu_legend_updater)

        # constructive_interference.add_updater(constructive_interference_updater)
        # destructive_interference.add_updater(destructive_interference_updater)

        # self.play(time_tracker.animate.set_value(26/c), rate_func = linear, run_time = 10)


# +++ off-section +++

        # constructive_interference.remove_updater(constructive_interference_updater)
        # destructive_interference.remove_updater(destructive_interference_updater)


# +++ PART C (set t0 = 26)

        # self.add(constructive_interference, destructive_interference)
        # self.add(interference_legend)
        # self.add(nu_legend)

        # time_tracker = ValueTracker(t0/c)
        # nu_tracker = ValueTracker(nu)

        # sound_waves.add_updater(sound_waves_updater)

        # self.play(time_tracker.animate.set_value(29/c), FadeOut(constructive_interference), FadeOut(destructive_interference), FadeOut(interference_legend), rate_func = linear, run_time = 3)
        # self.play(time_tracker.animate.set_value(39/c), rate_func = linear, run_time = 10)
        # self.wait(3)

Manim Community v0.18.1

In [ ]:
# 'ffmpeg -f concat -i ndw_interference_merge_list.txt -c copy ndw_interference_F1.mp4'
# 'ffmpeg -f concat -i ndw_interference_bg_blue_merge_list.txt -c copy ndw_interference_bg_blue_F2.mp4'